In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import io
import nltk
import json
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalAveragePooling1D, Flatten, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv1D, MaxPool1D

In [3]:
data = pd.read_json("/kaggle/input/chatbots-intent-recognition-dataset/Intent.json")

In [4]:
data

,intents
0,"{'intent': 'Greeting', 'text': ['Hi', 'Hi ther..."
1,"{'intent': 'GreetingResponse', 'text': ['My us..."
2,"{'intent': 'CourtesyGreeting', 'text': ['How a..."
3,"{'intent': 'CourtesyGreetingResponse', 'text':..."
4,"{'intent': 'CurrentHumanQuery', 'text': ['What..."
5,"{'intent': 'NameQuery', 'text': ['What is your..."
6,"{'intent': 'RealNameQuery', 'text': ['What is ..."
7,"{'intent': 'TimeQuery', 'text': ['What is the ..."
8,"{'intent': 'Thanks', 'text': ['OK thank you', ..."
9,"{'intent': 'NotTalking2U', 'text': ['I am not ..."


In [8]:
def json_data(data):
    tags = []
    patterns = []
    responses = {}
    for intent in data['intents']:
        responses[intent['intent']]= intent['responses']
        for lines in intent['text']:
            patterns.append(lines)
            tags.append(intent['intent'])
    return [tags, patterns, responses]

In [9]:
[tags, patterns, responses]= json_data(data)

In [10]:
df = pd.DataFrame({"patterns":patterns, "tags":tags})

In [11]:
df = df.sample(frac=1)

In [12]:
df.head()

,patterns,tags
7,My user is Adam,GreetingResponse
54,Can you tell me the time?,TimeQuery
84,fuck,Swearing
44,What is your real name please?,RealNameQuery
100,Do you see anyone in the camera?,WhoAmI


In [13]:
import re
r = re.compile(r'[^\w\s]+')
df["patterns"] = [r.sub('',s) for s in df["patterns"].tolist()]

In [14]:
df.head()

,patterns,tags
7,My user is Adam,GreetingResponse
54,Can you tell me the time,TimeQuery
84,fuck,Swearing
44,What is your real name please,RealNameQuery
100,Do you see anyone in the camera,WhoAmI


In [15]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(df['patterns'])
train = tokenizer.texts_to_sequences(df["patterns"])
features = pad_sequences(train)
le = LabelEncoder()
labels = le.fit_transform(df["tags"])

In [16]:
vocab = len(tokenizer.word_index)
output_len = le.classes_.shape[0]

In [19]:
model = Sequential()
model.add(Input(shape=(features.shape[1])))
model.add(Embedding(vocab + 1,100))
model.add(Conv1D(filters=32, kernel_size=5, activation="relu", kernel_initializer=tf.keras.initializers.GlorotNormal(),bias_regularizer=tf.keras.regularizers.L2(0.0001), kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))) 
model.add(Dropout(0.3))
model.add(LSTM(32, dropout=0.3,return_sequences=True))
model.add(LSTM(16, dropout=0.3,return_sequences=False))
model.add(Dense(128,activation="relu", activity_regularizer = tf.keras.regularizers.L2(0.0001))) 
model.add(Dropout(0.6))
model.add(Dense(output_len, activation="softmax", activity_regularizer = tf.keras.regularizers.L2(0.0001)))

In [21]:
#importing unsupervised learning algorithm
!wget https://nlp.stanford.edu/data/glove.6B.zip


--2023-04-13 13:07:56--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-13 13:07:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2023-04-13 13:10:35 (5.18 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [23]:
model.layers

In [27]:
!unzip glove.6B.zip.1

Archive:  glove.6B.zip.1
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [28]:
glove_dir = "glove.6B.100d.txt"
embeddings_index = {}
file_ = open(glove_dir)
for line in file_:
    arr = line.split()
    single_word = arr[0]
    w = np.asarray(arr[1:],dtype='float32')
    embeddings_index[single_word] = w
file_.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [36]:
max_words = vocab+ 1
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words,100)).astype(object)
for word , i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [40]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [43]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [44]:
from keras.callbacks import TensorBoard, EarlyStopping
earlyStopping = EarlyStopping(monitor = 'loss', patience = 400, mode = 'min', restore_best_weights = True)

In [45]:
history_training = model.fit(features,labels,epochs=1000, batch_size=64, callbacks=[ earlyStopping])

Epoch 1/1000
3/3 [==============================] - 5s 22ms/step - loss: 3.0982 - accuracy: 0.0490
Epoch 2/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0954 - accuracy: 0.0350
Epoch 3/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0891 - accuracy: 0.0350
Epoch 4/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.0826 - accuracy: 0.0629
Epoch 5/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0889 - accuracy: 0.0490
Epoch 6/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0755 - accuracy: 0.0909
Epoch 7/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.0748 - accuracy: 0.0559
Epoch 8/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.0708 - accuracy: 0.1329
Epoch 9/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0539 - accuracy: 0.0909
Epoch 10/1000
3/3 [==============================] - 0s 12ms/step - loss: 3.0598 - accuracy: 0.0839
Epoch 11/

In [58]:
import random
def generate_answer(query):
    texts = []
    pred_input = query
    pred_input = [letters.lower() for letters in pred_input if letters not in string.punctuation]
    pred_input = ''.join(pred_input)
    texts.append(pred_input)
    pred_input = tokenizer.texts_to_sequences(texts)
    pred_input = np.array(pred_input).reshape(-1)
    pred_input = pad_sequences([pred_input],features.shape[1])
    output = model.predict(pred_input)
    output = output.argmax()
    response_tag = le.inverse_transform([output])[0]
    return random.choice(responses[response_tag])

In [59]:
print("Bot: Hello! I am here to chat. if you need to go just say \"bye\" :)")
person="anything"
bye= ["bye", "stop", "leave","goodbye", "good bye", "see you later", "exit","quit"]
while person not in bye:
    print("you:", end=" ")
    person = input()
    res_tag = generate_answer(person)
    print("Bot: "+res_tag)

Bot: Hello! I am here to chat. if you need to go just say "bye" :)
you: 

 hi


1/1 [==============================] - 0s 26ms/step
Bot: Hi human, please tell me your GeniSys user
you: 

 it is silvia


1/1 [==============================] - 0s 25ms/step
Bot: Good! Hi <HUMAN>, how can I help you?
you: 

...

1/1 [==============================] - 0s 28ms/step
Bot: Well I would not be a very clever AI if I did not would I?
you: 

 yes


1/1 [==============================] - 0s 29ms/step
Bot: I do in deed!
you: 

 bye


1/1 [==============================] - 0s 28ms/step
Bot: See you later
